In [1]:
import base64
import requests
import json
import pandas as pd

User = ""
Password = ""

headers = {"Authorization": "Basic " + str(base64.b64encode(bytes(User + ":" + Password,'utf-8')).decode('utf-8'))}

def toHtmlQuery_(**kwargs):
    """Возвращает строку для загрузки в api"""
    resultstr = ""
    for key, value in kwargs.items():
        if resultstr=="":
            resultstr = "?"+key+"="+str(value)
        else:
            resultstr = resultstr+"&"+key+"="+str(value)
    return resultstr

def get_data(url,params={'':''}):
    """Получает Json объект по api запросу"""
    encParams = toHtmlQuery_(**params)
    response = requests.get(url+encParams, headers=headers)
    return response.json()

def offset_params(offset_num:int,params:dict)->dict:
    """меняет параметр в словаре params"""
    params['offset'] = params['limit']*offset_num
    return params




# Необходимо получить список всех покупателей в заданный период 

In [2]:
class CounterParty():
    def __init__(self, date_start,date_finish):
        """формат даты 2021-01-01"""
        self.date_start = date_start
        self.date_finish = date_finish
        self.api_method_counterparty = "https://online.moysklad.ru/api/remap/1.2/report/profit/bycounterparty"
        self.params_profit = {
                "limit": 1000, 
                "momentFrom":self.date_start+" 00:00:00",
                "momentTo":  self.date_finish+" 23:59:00"} 
        
        
    def get_counterparty_json(self, num_circle):
        params = offset_params(0,self.params_profit)
        data =  get_data(self.api_method_counterparty,params)['rows']
        return data
    
    def get_all_counterparty_json(self):
        data_counterparty_json = []
        for i in range(0,100):
            print(i)   
            data = self.get_counterparty_json(i)
            
            if len(data) ==0:
                return data_counterparty_json

            data_counterparty_json += data
            return data_counterparty_json
        
    def get_list_counterparty(self):
        data_json_counterparty = self.get_all_counterparty_json()
        list_counterparty = []
        
        for num,elem in enumerate(data_json_counterparty):
            list_counterparty.append(
            [elem['counterparty']['meta']['href'],
             elem['counterparty']['name']])
        return list_counterparty
        

# Выгрузка отчета по прибыльности по товарам по контрагенту

In [3]:
class ReportData():
    def __init__(self,counterpartyid,date_start,date_finish):
        self.date_start = date_start
        self.date_finish = date_finish
        self.counterpartyid = counterpartyid
        self.api_methon_get_data = "https://online.moysklad.ru/api/remap/1.2/report/profit/byproduct"
        self.params_profit = {
                "limit": 1000, 
                "momentFrom":self.date_start+" 00:00:00",
                "momentTo":  self.date_finish+" 23:59:00",
                "filter": "counterparty="+counterpartyid} 
        
        
    def get_report_data_json(self, num_circle):
        params = offset_params(0,self.params_profit)
        data =  get_data(self.api_methon_get_data,params)['rows']
        return data
    
    def get_all_report_data_json(self):
        data_counterparty_json = []
        for i in range(0,100):
            print(i,end = ', ')   
            data = self.get_report_data_json(i)
            
            if len(data) ==0:
                return data_counterparty_json

            data_counterparty_json += data
            return data_counterparty_json
        

# Совмещение отчетов и обработка

In [115]:
date_start = "2021-12-01"

In [116]:
def finish_month(date:str):
    """формат 2021-01-01"""
    from datetime import timedelta
    import datetime
    
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    finish_date = datetime.date(dt.year,dt.month+1,1)-timedelta(days=1)
    return finish_date.strftime("%Y-%m-%d")

In [118]:
#date_finish = finish_month(date_start)
date_finish ="2021-12-31"

In [119]:
date_finish

'2021-12-31'

In [120]:
def parse_couterparty(json_couterparty_data,name_counterparty,href_counterparty):
    """промежуточная функция чтобы распарсить полученный JSON 
    объект в словарь"""
    data_contractor_sale = []
    for elem in json_couterparty_data:
        if 'article' in elem['assortment']:
            article = elem['assortment']['article']
        else:
            article='---'

        data_contractor_sale.append({
            'Name_counterparty':name_counterparty,
            'href_counterparty':href_counterparty,
            'Date':date_start,
            'Name':elem['assortment']['name'],
            'Code':elem['assortment']['code'],
            'Article':article,
            'SellQuantity':elem['sellQuantity'],
            'SellCostSum':elem['sellCostSum'],
            'SellSum':elem['sellSum'],
            'returnQuantity':elem['returnQuantity'],
            'returnPrice':elem['returnPrice'],
            'returnCost':elem['returnCost'],
            'returnSum':elem['returnSum']
        })
        
    return data_contractor_sale

In [121]:
def get_tag_counterparty(href_couterparty):
    api_method = href_couterparty
    data = get_data(api_method)
    
    if 'tags' in data:
        return data['tags']
    else:
        return '---'

In [122]:
CP_obj = CounterParty(
    date_start = date_start,
    date_finish = date_finish,
)

In [123]:
list_counterpatry = CP_obj.get_list_counterparty()

0


In [124]:
date_all_data = []

In [125]:
for num,counterparty in enumerate(list_counterpatry):
    RD_obj = ReportData(
        date_start = date_start,
        date_finish = date_finish,
        counterpartyid = counterparty[0],
    )
    
    data_counterparty = RD_obj.get_all_report_data_json()
    data_counterparty_1 = parse_couterparty(data_counterparty,
                                           name_counterparty = counterparty[1],
                                           href_counterparty = counterparty[0])
    
    date_all_data.extend(data_counterparty_1)
    print(num,end = ', ')

0, 0, 0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0, 6, 0, 7, 0, 8, 0, 9, 0, 10, 0, 11, 0, 12, 0, 13, 0, 14, 0, 15, 0, 16, 0, 17, 0, 18, 0, 19, 0, 20, 0, 21, 0, 22, 0, 23, 0, 24, 0, 25, 0, 26, 0, 27, 0, 28, 0, 29, 0, 30, 0, 31, 0, 32, 0, 33, 0, 34, 0, 35, 0, 36, 0, 37, 0, 38, 0, 39, 0, 40, 0, 41, 0, 42, 0, 43, 0, 44, 0, 45, 0, 46, 0, 47, 0, 48, 0, 49, 0, 50, 0, 51, 0, 52, 0, 53, 0, 54, 0, 55, 0, 56, 0, 57, 0, 58, 0, 59, 0, 60, 0, 61, 0, 62, 0, 63, 0, 64, 0, 65, 0, 66, 0, 67, 0, 68, 0, 69, 0, 70, 0, 71, 0, 72, 0, 73, 0, 74, 0, 75, 0, 76, 0, 77, 0, 78, 0, 79, 0, 80, 0, 81, 0, 82, 0, 83, 0, 84, 0, 85, 0, 86, 0, 87, 0, 88, 0, 89, 0, 90, 0, 91, 0, 92, 0, 93, 0, 94, 0, 95, 0, 96, 0, 97, 0, 98, 0, 99, 0, 100, 0, 101, 0, 102, 0, 103, 0, 104, 0, 105, 0, 106, 0, 107, 0, 108, 0, 109, 0, 110, 0, 111, 0, 112, 0, 113, 0, 114, 0, 115, 0, 116, 0, 117, 0, 118, 0, 119, 0, 120, 0, 121, 0, 122, 0, 123, 0, 124, 0, 125, 0, 126, 0, 127, 0, 128, 0, 129, 0, 130, 0, 131, 0, 132, 0, 133, 0, 134, 0, 135, 0, 136, 0, 137, 0, 138

In [126]:
df_counterparty = pd.DataFrame(date_all_data)

In [127]:
list_counterparty_unique = df_counterparty.href_counterparty.unique()

In [128]:
data_tags = []

In [129]:
for counterparty_href in list_counterparty_unique:
    data_tags.append({
        'href_counterparty':counterparty_href,
        'tags':', '.join(get_tag_counterparty(counterparty_href))
        
    })

In [130]:
df_tags = pd.DataFrame(data_tags)

In [131]:
df_total = df_counterparty.merge(right = df_tags,how = 'left',on = 'href_counterparty')

In [132]:
df_total.head(2)

,Name_counterparty,href_counterparty,Date,Name,Code,Article,SellQuantity,SellCostSum,SellSum,returnQuantity,returnPrice,returnCost,returnSum,tags
0,Абдукаюмова Наргис Бахтиеровна (@detmir_tj),https://online.moysklad.ru/api/remap/1.2/entit...,2021-12-01,Музыкальная игрушка Умный зайка alilo R1. Цвет...,6954644609072,60907,10.0,977709.0,1727307.0,0.0,0.0,0.0,0.0,"04 интернет-магазин, 01 клиенты, душанбе, 02 т..."
1,Абдукаюмова Наргис Бахтиеровна (@detmir_tj),https://online.moysklad.ru/api/remap/1.2/entit...,2021-12-01,Музыкальная игрушка Умный зайка alilo R1. Цвет...,6954644609089,60908,5.0,488920.0,863653.5,0.0,0.0,0.0,0.0,"04 интернет-магазин, 01 клиенты, душанбе, 02 т..."


In [133]:
df_total['status_counterpatry'] = df_total['tags'].apply(lambda x: 'Клиент' if '01 клиенты' in x else 'Остальное')

#  Загрузка данных в MySQL

In [134]:
list_values = ['SellCostSum','SellSum','returnPrice','returnCost','returnSum']

for col in list_values:
    df_total[col] = df_total[col]/100

In [135]:
df_total.head(2)

,Name_counterparty,href_counterparty,Date,Name,Code,Article,SellQuantity,SellCostSum,SellSum,returnQuantity,returnPrice,returnCost,returnSum,tags,status_counterpatry
0,Абдукаюмова Наргис Бахтиеровна (@detmir_tj),https://online.moysklad.ru/api/remap/1.2/entit...,2021-12-01,Музыкальная игрушка Умный зайка alilo R1. Цвет...,6954644609072,60907,10.0,9777.09,17273.070,0.0,0.0,0.0,0.0,"04 интернет-магазин, 01 клиенты, душанбе, 02 т...",Клиент
1,Абдукаюмова Наргис Бахтиеровна (@detmir_tj),https://online.moysklad.ru/api/remap/1.2/entit...,2021-12-01,Музыкальная игрушка Умный зайка alilo R1. Цвет...,6954644609089,60908,5.0,4889.20,8636.535,0.0,0.0,0.0,0.0,"04 интернет-магазин, 01 клиенты, душанбе, 02 т...",Клиент


In [136]:
import sqlalchemy
import pandas as pd

database_username = ''
database_password = ''
database_ip       = ''
database_name     = ''
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [137]:
name_table = 'MS_report_total'

In [138]:
def join_table(name_table,data):
    name_temp = name_table+'_temp'
    for i in range(0,len(data),50):
        with database_connection.begin() as connection:
            data.iloc[i:i+50].to_sql(con=connection, name=name_temp, if_exists='append')
            if i%1000==0:
                print(i, end=', ')
                
    SQL_DROP = SQL_DROP = f"""DELETE FROM {name_table} 
    where (Date>=(SELECT MIN(Date) FROM {name_temp}) AND 
          Date<=(SELECT MAX(Date) FROM {name_temp}))"""
    
    result = database_connection.execute(SQL_DROP)
    result = database_connection.execute(f"INSERT INTO {name_table} SELECT * FROM {name_temp}")
    result = database_connection.execute(f"DROP TABLE {name_temp}")

In [139]:
join_table(name_table,df_total)

0, 1000, 2000, 3000, 

In [140]:
database_connection.dispose()

In [141]:
print("OK",date_start)

OK 2021-12-01


Вариант когда нету таблицы

In [30]:
# def insert_table(name_table,data):
#     for i in range(0,len(data),50):
#         with database_connection.begin() as connection:
#             data.iloc[i:i+50].to_sql(con=connection, name=name_table, if_exists='append')
#             if i%1000==0:
#                 print(i, end=', ')

In [331]:
# insert_table(name_table,df_total)

In [275]:
# database_connection.dispose()